## Dependencies

In [ ]:
# Standard Pytorch Libraries
import torch
import torch.nn as nn
import torch.optim as optim

# RF from sklearn
from sklearn.ensemble import RandomForestRegressor

## Neural Network with 1 hidden layer

In [ ]:
# Simple MLP (Multi-Layer Perceptron)
class ScalingFactorPredictor(nn.Module):

  def __init__(self, input_size, hidden_size, output_size):

    super().__init__()
    self.fc1 = nn.Linear(input_size, hidden_size)  # First layer
    self.relu = nn.ReLU()                          # Activation function (can try different ones)
    self.fc2 = nn.Linear(hidden_size, output_size) # Output layer

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

## Random Forest

In [ ]:
# Random forest regression model
rf_reg = RandomForestRegressor(n_estimators=100)

## Training the Neural Network

In [ ]:
input_size = 1     # Input features (probability p)
hidden_size = 4    # Number of neurons in the hidden layer
output_size = 1    # Output features (scaling factor)

model = SimpleNN(input_size, hidden_size, output_size)

# Loss and optimizer
criterion = nn.MSELoss()  # Mean Squared Error Loss Function
optimizer = optim.Adam(model.parameters(), lr=0.001)  # GD Optimizer

epochs = 3
for epoch in range(epochs):

    # Forward pass
    outputs = model(<input_data>)
    loss = criterion(outputs, y)

    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

## Train the Random Forest:

In [ ]:
rf.fit(<x_train>, <y_train>)